# Hyperparameter optimization

## Import packages

In [18]:
from rdkit import RDLogger
import logging
import warnings

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
RDLogger.DisableLog('rdApp.*')

from sklearn.metrics import accuracy_score
from deepmol.metrics import Metric
from sklearn.svm import SVC
from deepmol.parameter_optimization import HyperparameterOptimizerValidation

from deepmol.splitters import RandomSplitter
from deepmol.compound_featurization import MorganFingerprint
from deepmol.loaders import SDFLoader


from tensorflow.keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers

## First of all, let's load the data

In [2]:
dataset = SDFLoader("../data/CHEMBL217_conformers.sdf", id_field="_ID", labels_fields=["_Class"]).create_dataset()
train_dataset, valid_dataset, test_dataset = RandomSplitter().train_valid_test_split(dataset, frac_train=0.8, frac_valid=0.1, frac_test=0.1)

2023-06-01 20:13:37,282 — INFO — Assuming classification since there are less than 10 unique y values. If otherwise, explicitly set the mode to 'regression'!


## Let's featurize the data

In [3]:
morgan_fingerprints = MorganFingerprint()
morgan_fingerprints.featurize(train_dataset, inplace=True)
morgan_fingerprints.featurize(valid_dataset, inplace=True)
morgan_fingerprints.featurize(test_dataset, inplace=True)

## Hyperparameter tuning with scikit-learn

DeepMol provide methods to perform hyperparameter tuning with scikit-learn models. The hyperparameter tuning can be performed with a validation set, previously created, or with cross validation. Anyway, the hyperparameter tuning is performed with a random search if the number of iterations is specified, otherwise a grid search is performed.

Moreover, for each method the user have to specify the metric to optimize and if the metric has to be maximized or minimized. The user must specify the parameters to optimize and the values to try for each parameter.

The parameters must be specified as a dictionary, where the keys are the parameters names and the values are the values to try.

Let's see how to perform hyperparameter tuning of a SVM with a validation set.

In [ ]:
params_dict_svc = {"C": [1.0, 1.2, 0.8], "kernel": ['linear', 'poly', 'rbf']} # The keys are the parameters names and the values are the values to try
optimizer = HyperparameterOptimizerValidation(SVC)
best_svm, best_hyperparams, all_results = optimizer.hyperparameter_search(train_dataset=train_dataset,
                                                                          valid_dataset=valid_dataset,
                                                                          metric=Metric(accuracy_score),
                                                                          maximize_metric=True,
                                                                          n_iter_search=2,
                                                                          params_dict=params_dict_svc,
                                                                          )

In the end, we can check the performance of the best model on the test set.

In [5]:
best_svm.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

({'accuracy_score': 0.9879735417919423}, {})

We can also check the best combination of hyperparameters found.

In [6]:
best_hyperparams

{'C': 1.0, 'kernel': 'poly'}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [7]:
all_results

{'_C_1.000000_kernel_poly': 0.9885679903730445,
 '_C_0.800000_kernel_linear': 0.9765342960288809}

Finally, save your best model for deployment and new predictions!

In [8]:
best_svm.save("my_model")

Bring it back to life and make predictions!

In [9]:
from deepmol.models import SklearnModel

SklearnModel.load("my_model").predict(test_dataset)

array([0., 0., 1., ..., 1., 0., 0.])

## Now let's try with cross validation

In [16]:
from deepmol.parameter_optimization import HyperparameterOptimizerCV

params_dict_svc = {"C": [1.0, 1.2, 0.8], "kernel": ['linear', 'poly', 'rbf']}
optimizer = HyperparameterOptimizerCV(SVC)
best_svm, best_hyperparams, all_results = optimizer.hyperparameter_search(train_dataset=train_dataset,
                                                                          metric=Metric(accuracy_score),
                                                                          maximize_metric=True,
                                                                          cv=3,
                                                                          n_iter_search=2,
                                                                          params_dict=params_dict_svc,
                                                                          model_type="sklearn"
                                                                          )

2023-06-01 16:19:58,650 — INFO — MODEL TYPE: sklearn
2023-06-01 16:19:58,651 — INFO — Fitting 2 random models from a space of 9 possible models.
2023-06-01 16:21:17,213 — INFO — 
 
 Best <function accuracy_score at 0x7f820037f0d0>: 0.973680 using {'kernel': 'linear', 'C': 1.0}
2023-06-01 16:21:17,214 — INFO — 
 <function accuracy_score at 0x7f820037f0d0>: 0.973304 (0.000281) with: {'kernel': 'linear', 'C': 1.2} 

2023-06-01 16:21:17,214 — INFO — 
 <function accuracy_score at 0x7f820037f0d0>: 0.973680 (0.000282) with: {'kernel': 'linear', 'C': 1.0} 

2023-06-01 16:21:17,215 — INFO — Fitting best model!
2023-06-01 16:21:32,759 — INFO — SklearnModel(mode='classification', model=SVC(kernel='linear'),
             model_dir='/tmp/tmpom34y1bo')


Then, we can check the performance of the best model on the test set.

In [11]:
best_svm.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

({'accuracy_score': 0.9879735417919423}, {})

We can also check the best combination of hyperparameters found.

In [12]:
best_hyperparams

{'probability': True, 'kernel': 'poly', 'C': 1.2}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [13]:
all_results

{'mean_fit_time': array([103.76487271, 104.18803382]),
 'std_fit_time': array([1.19065455, 0.81880114]),
 'mean_score_time': array([8.71299458, 8.91998561]),
 'std_score_time': array([0.4336231 , 0.08003275]),
 'param_probability': masked_array(data=[True, True],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['poly', 'poly'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[1.2, 1.0],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'probability': True, 'kernel': 'poly', 'C': 1.2},
  {'probability': True, 'kernel': 'poly', 'C': 1.0}],
 'split0_test_score': array([0.9835326 , 0.98217911]),
 'split1_test_score': array([0.98578841, 0.98511166]),
 'split2_test_score': array([0.98285199, 0.98217509]),
 'mean_test_score': array([0.98405766, 0.98315529]),
 'std_test_score': array([0.00125497, 0.00138

Finally, save your best model for deployment and new predictions!

In [14]:
best_svm.save("my_model")

Bring it back to life and make predictions!

In [15]:
from deepmol.models import SklearnModel

SklearnModel.load("my_model").predict(test_dataset)

array([[9.99798926e-01, 2.01074317e-04],
       [9.99341488e-01, 6.58511735e-04],
       [3.02237487e-03, 9.96977625e-01],
       ...,
       [1.68278370e-08, 9.99999983e-01],
       [9.90501082e-01, 9.49891777e-03],
       [9.99827191e-01, 1.72809148e-04]])

# Hyperparameter tuning with keras

DeepMol provide methods to perform hyperparameter tuning with keras models. The hyperparameter tuning can be performed with a validation set, previously created, or with cross validation. Anyway, the hyperparameter tuning is performed with a random search if the number of iterations is specified, otherwise a grid search is performed.

As explained in the models section, to create a Keras model one have to define a function with the model architecture and the parameters to optimize.

In [16]:
def create_model(input_dim, optimizer='adam', dropout=0.5):
    # create model
    inputs = layers.Input(shape=input_dim)

    # Define the shared layers
    shared_layer_1 = layers.Dense(64, activation="relu")
    dropout_1 = Dropout(dropout)
    shared_layer_2 = layers.Dense(32, activation="relu")

    # Define the shared layers for the inputs
    x = shared_layer_1(inputs)
    x = dropout_1(x)
    x = shared_layer_2(x)

    task_output = layers.Dense(1, activation="sigmoid")(x)

    # Define the model that outputs the predictions for each task
    model = keras.Model(inputs=inputs, outputs=task_output)
    # Compile the model with different loss functions and metrics for each task
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model


Let's see how to perform hyperparameter tuning of a DNN with a validation set.

In [17]:
from tensorflow.keras.optimizers import Adam

optimizer = HyperparameterOptimizerValidation(create_model)
params_dict_dense = {
                   "input_dim": [train_dataset.X.shape[1]],
                   "dropout": [0.5, 0.6, 0.7],
                   "optimizer": [Adam]
                   }

best_dnn, best_hyperparams, all_results = optimizer.hyperparameter_search(train_dataset=train_dataset,
                                                                          valid_dataset=valid_dataset,
                                                                          metric=Metric(accuracy_score),
                                                                          maximize_metric=True,
                                                                          n_iter_search=2,
                                                                          params_dict=params_dict_dense,
                                                                          model_type="keras"
                                                                          )

2023-06-01 20:19:47,721 — INFO — Fitting 2 random models from a space of 3 possible models.
2023-06-01 20:19:47,722 — INFO — Fitting model 1/2
2023-06-01 20:19:47,724 — INFO — hyperparameters: {'input_dim': 2048, 'dropout': 0.6, 'optimizer': <class 'keras.optimizers.adam.Adam'>}


2023-06-01 20:19:48.921162: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-06-01 20:19:48.959148: E tensorflow/compiler/xla/status_macros.cc:57] INTERNAL: RET_CHECK failure (tensorflow/compiler/xla/service/gpu/gpu_compiler.cc:618) dnn != nullptr 
*** Begin stack trace ***
	tsl::CurrentStackTrace[abi:cxx11]()
	
	xla::status_macros::MakeErrorStream::Impl::GetStatus()
	xla::gpu::GpuCompiler::OptimizeHloModule(xla::HloModule*, stream_executor::StreamExecutor*, stream_executor::DeviceMemoryAllocator*, xla::gpu::GpuTargetConfig const&, xla::AutotuneResults const*)
	xla::gpu::GpuCompiler::RunHloPasses(std::unique_ptr<xla::Hl

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_4' defined at (most recent call last):
    File "/home/joao/miniconda3/envs/deepmol/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/joao/miniconda3/envs/deepmol/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/joao/miniconda3/envs/deepmol/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/joao/miniconda3/envs/deepmol/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/joao/miniconda3/envs/deepmol/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/joao/miniconda3/envs/deepmol/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever

In the end, we can check the performance of the best model on the test set.

In [8]:
best_dnn.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

({'accuracy_score': 0.9771497294046904}, {})

We can also check the best combination of hyperparameters found.

In [9]:
best_hyperparams

{'input_dim': 2048, 'dropout': 0.7, 'optimizer': 'adam'}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [10]:
all_results

{'_dropout_0.600000_input_dim_2048_optimizer_rmsprop': 0.97352587244284,
 '_dropout_0.700000_input_dim_2048_optimizer_adam': 0.9753309265944645}

In [12]:
best_dnn

SklearnModel(mode='classification',
             model=<tensorflow.python.keras.engine.functional.Functional object at 0x7f03472e95e0>,
             model_dir='/tmp/tmpc7tn1hyw')

Finally, save your best model for deployment and new predictions!

In [11]:
best_dnn.save("my_model")

TypeError: cannot pickle '_thread._local' object

# Hyperparameter tuning with deepchem models with cross validation

In [3]:
from deepmol.parameter_optimization import HyperparameterOptimizerCV
from deepmol.compound_featurization import ConvMolFeat
from sklearn.metrics import roc_auc_score, precision_score
from deepmol.models import DeepChemModel
from deepchem.models import GraphConvModel

ConvMolFeat().featurize(train_dataset)
ConvMolFeat().featurize(valid_dataset)
ConvMolFeat().featurize(test_dataset)

def graphconv_builder(graph_conv_layers, batch_size=256, epochs=5):
    graph = GraphConvModel(n_tasks=1, graph_conv_layers=graph_conv_layers, batch_size=batch_size,
                           mode='classification')
    return DeepChemModel(graph, epochs=epochs)

model_graph = HyperparameterOptimizerCV(model_builder=graphconv_builder)

best_model, _, _ = model_graph.hyperparameter_search(train_dataset=train_dataset,
                                                     metric=Metric(roc_auc_score),
                                                     n_iter_search=2,
                                                     maximize_metric=True,
                                                     cv = 2,
                                                     params_dict={'graph_conv_layers': [[64, 64], [32, 32]]},
                                                     model_type="deepchem")


2023-03-17 18:46:09,927 — INFO — MODEL TYPE: deepchem
2023-03-17 18:46:09,927 — INFO — Computing Stratified K-fold split


2023-03-17 18:46:12.276353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-17 18:46:12.276372: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-17 18:46:12.276385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bisbii): /proc/driver/nvidia/version does not exist
2023-03-17 18:46:12.276531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-03-17 18:47:14,884 — INFO — 
 
 Best <deepmol.metrics.metrics.Metric object at 0x7fbf2be4cd00>: 0.977278 using {'graph_conv_layers': [64, 64]}
2023-03-17 18:47:14,885 — INFO — 
 <deepmol.metrics.metrics.Metric object at 0x7fbf2be4cd00>: 0.977278 (0.002598) with: {'graph_conv_layers': [64, 64]} 

2023-03-17 18:47:14,885 — INFO — 
 <deepmol.metrics.metrics.Metric object at 0x7fbf2be4cd00>: 0.965922 (0.002891) with: {'graph_conv_layers': [32, 32]} 



In [12]:
test_preds = best_model.predict(test_dataset)

metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score)]

best_model.evaluate(test_dataset, metrics)


({'roc_auc_score': 0.9898796965934775,
  'precision_score': 0.9402480270574972,
  'accuracy_score': 0.9591100420926038},
 {})